In [1]:
from qurious.rl.environments.mongodb.mongodb_environment import MongoDBEnvironment

env = MongoDBEnvironment("mongodb://localhost:27017", "qurious", "points_2d")
env._create_2d_dataset(100000)

docs = list(env.collection.find())
for d in docs[:5]:
    print(d)
count = env.collection.count_documents({})
print(f"\n... {count} documents in the collection ...")

env.indexes

{'_id': 0, 'x': 0, 'y': 75}
{'_id': 1, 'x': 23, 'y': 76}
{'_id': 2, 'x': 62, 'y': 69}
{'_id': 3, 'x': 20, 'y': 99}
{'_id': 4, 'x': 99, 'y': 40}

... 100000 documents in the collection ...


{'x_1': [('x', 1)],
 'y_1': [('y', 1)],
 'x_1_y_1': [('x', 1), ('y', 1)],
 'COLL_SCAN': [('$natural', 1)]}

In [2]:
state = env.get_state()
print(f"state: {state}")

next_step, reward, done, info = env.step("x_1")

print(f"next step: {next_step}")
print(f"reward: {reward}")
print(f"done: {done}")
print(f"info: {info}")


state: None
next step: {'x': {'$gte': 36, '$lte': 54}, 'y': {'$gte': 11, '$lte': 27}}
reward: -47
done: True
info: {'explain': {'executionSuccess': True, 'nReturned': 100000, 'executionTimeMillis': 47, 'totalKeysExamined': 100000, 'totalDocsExamined': 100000, 'executionStages': {'isCached': False, 'stage': 'FETCH', 'nReturned': 100000, 'executionTimeMillisEstimate': 43, 'works': 100001, 'advanced': 100000, 'needTime': 0, 'needYield': 0, 'saveState': 2, 'restoreState': 2, 'isEOF': 1, 'docsExamined': 100000, 'alreadyHasObj': 0, 'inputStage': {'stage': 'IXSCAN', 'nReturned': 100000, 'executionTimeMillisEstimate': 2, 'works': 100001, 'advanced': 100000, 'needTime': 0, 'needYield': 0, 'saveState': 2, 'restoreState': 2, 'isEOF': 1, 'keyPattern': {'x': 1}, 'indexName': 'x_1', 'isMultiKey': False, 'multiKeyPaths': {'x': []}, 'isUnique': False, 'isSparse': False, 'isPartial': False, 'indexVersion': 2, 'direction': 'forward', 'indexBounds': {'x': ['[MinKey, MaxKey]']}, 'keysExamined': 100000, 's

In [3]:
import random

for i in range(1000):
    state = env.get_state()
    action = random.choice(env.action_space)
    next_step, reward, done, info = env.step(action)
    # print(f"state={state}, action={action}, next_step={next_step}, reward={reward}, done={done}, info={info}")


In [ ]:
import numpy as np

from qurious.rl.policies.nn import DiscreteNNPolicy


def feature_extractor(state):
    """state is a MongoDB query with x and y fields and range queries."""
    x_min = state["x"]["$gte"]
    x_max = state["x"]["$lte"]
    y_min = state["y"]["$gte"]
    y_max = state["y"]["$lte"]
    return np.array([x_min, x_max, y_min, y_max])


policy = DiscreteNNPolicy(input_dim=2, n_actions=env.n_actions, feature_extractor=feature_extractor)

state = {"x": {"$gte": 0, "$lte": 10}, "y": {"$gte": 50, "$lte": 60}}
features = feature_extractor(state)
features

using device: mps


array([ 0, 10, 50, 60])